In [ ]:
def load_file(filename):
    average_attention_matrix_file = "/piech/u/joetey/multimodal-reasoning/results/files/results_geometry_3k_baseline_cot_20240601_194854/attention_weights/0.pt"
    average_attention_matrix = torch.load(average_attention_matrix_file, map_location=torch.device('cpu'))
    attention_file = "/piech/u/joetey/multimodal-reasoning/results/files/results_geometry_3k_baseline_cot_20240601_194854/attention/0.json"
    with open(attention_file, "r") as file:
        attention_description = json.load(file)

    image_file = '/piech/u/joetey/multimodal-reasoning/dataset/eval/geometry_3k/images/' + attention_description["image_file"].split("/")[-1]
    output_tokens = [token.replace('Ġ', '').replace('Ċ', '') for token in attention_description["output_tokens"]]
    input_tokens = [token.replace('Ġ', '').replace('Ċ', '') for token in attention_description["input_tokens"]]
    
    image_token_indices = [i for i, token in enumerate(input_tokens) if token.startswith("image_")]
    x_image_token_start = image_token_indices[0]
    x_image_token_end = image_token_indices[-1] + 1
    main_input_text = input_tokens[x_image_token_end + 1:len(input_tokens)]

In [ ]:
from matplotlib.colors import Normalize

def calculate_attention_values(attention_matrix, input_tokens, output_tokens, image_token_start_idx, image_token_end_idx):
    # Calculate the average image tokens for each output token before normalizing
    average_attention_scores = []

    # calculate rationale's attention scores
    for i, output_token in enumerate(output_tokens):
        output_relevancy = attention_matrix[len(input_tokens) - 1 + i, :]
        image_tokens = output_relevancy[image_token_start_idx:image_token_end_idx]
        text_tokens = output_relevancy[image_token_end_idx + 1:len(input_tokens)-3]
    
        average_text_tokens = np.mean(text_tokens)
        average_image_tokens = np.mean(image_tokens)
        
        average_attention_scores.append({
            "id": i,
            "output_token": output_token,
            "average_text": average_text_tokens,
            "average_image_tokens": average_image_tokens
        })

    return average_attention_scores 

def calculate_image_text_ratio(attention_scores, normalize=True):
    ratio_list = []
    for output_token in attention_scores:
        ratio_list.append(output_token["average_image_tokens"] / output_token["average_text"])

    ratio_list = np.array(ratio_list)
    
    return np.max(ratio_list)